<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" width=500></td>
<img src="https://www.zigobot.ch/images/stories/virtuemart/product/Thymio_II_5288c11c8c241.jpg" width=500>
# <center>Project of Basics of mobile robotics</center>
## <center> Broccard Brendan, Ferreira Lopes Filipe, Pillet Maxime, Schramm Arthur</center>
<hr style="clear:both">
<p style="font-size:0.85em; margin:2px; text-align:justify">
This Juypter notebook will be the report of our project as part of the course "Basics of mobile robotics" given by Prof. Francesco Mondada.</p>
<hr style="clear:both">

## 1. Introduction
<p>$\;\;\;\;\;\;$ In this project, the goal was to create an environment in which we would have a starting point, a goal to reach and multiple obstacles in the way. We implented the following code on our Thymio robot in order to make it go from the starting point to the goal, following the most efficient path through the initial obstacles and using its sensors to avoid sudden obstacles on its way. A camera allows us to create a map of the environment at the beginning of the program and further on to detect the robot's position and orientation. That information is first filtered with the Kalman filter and then processed by a PD controller that we created in order to make Thymio go smoothly in direction of it's desired goal. Thanks to the Kalman filter, if the camera is suddenly obstruated, we can still estimate the robot's position and orientation. We update our values in a main "while" loop that occurs every 0.1 second.
<br>
<br>$\;\;\;\;\;\;$ This is a typical map pattern with the thymio at the start, a local obstacle on its way, the stop in green and the obstacles in black : 
    
<img src="Images/map.jpeg" alt="Drawing" style="width: 500px;"/>

</p>

## 2. Details on each section of the project

### 2.1 Vision
<p>$\;\;\;\;\;\;$ In this section, we will explain how we implemented the vision to our project. The goal is to understand the map and to get the position of the start and stop point just as the position of the angles of our obstacles. 
<br>$\;\;\;\;\;\;$ The first step of the vision is a "while" loop at the beginning of our process, even before the Thymio is placed on the map. The loop is asked to detect at least the start and stop chips which is the condition to stop executing the loop. The chips are differientiated by their HSV color code, the start is blue and the finish green. It also checks if we have initial obstacles in our circuit and computes their angles positions. We consider the angles shifted by a security margin to consider Thymio's width, as if we widen our obstacles to ensure their good avoidance.
<br>
<br>$\;\;\;\;\;\;$ This first vision step gives us a map with our shifted angles : 

<img src="Images/Vision_avec_legende.png" alt="Drawing" style="width: 500px;"/>

<br>$\;\;\;\;\;\;$ The second step is to detect the position and the facing angle of Thymio after we placed it on the start chip. It is continuously computed since it is updated in our main loop. To do so, we pasted a yellow chip in the front of Thymio and a red one in the back. The robot's position is considered as being in the middle of the two chips. And the yellow chip, coupled with the red one, allows to have the orientation of Thymio. 
<br>
<br> $\;\;\;\;\;\;$The robot's position is given by the blue circle and its orientation is given by the green line :
<br>
<img src="Images/Thymio_orientation.png" alt="Drawing" style="width: 150px;"/> 

### 2.2 Global Navigation
<p>$\;\;\;\;\;\;$ The global navigation computes the shortest path to go from a starting point to a finish point while going through our obstacle field. 
<br>$\;\;\;\;\;\;$ We chose to use the visibility graph as the pathfinding algorithm. It consists of a web of every possible routes to go from a point A to a point B passing by the angles of obstacles. Next, it calculates the shortest euclidian route and considers it as the optimized path to take. This path is given in the form of an array of points to reach in the given order.  
<br> 
<br>$\;\;\;\;\;\;$ The visibility graph, when applied to our map, returns this path :
<img src="Images/VisGraph.png" alt="Drawing" style="width: 500px;"/>

### 2.3 Filtering
<p>$\;\;\;\;\;\;$ Since the camera's datas are uncertain (due to noise or lack of precision), we use the Kalman filter based on theses datas in order to get a good estimation of the position and orientation of the robot (which are the states of the Thymio that we calculate at each time step). In order to estimate the new state, we use the last position, oriention and speed of the motors (linear and rotation). Then, Thymio's states are update, based on the last measurements, only if the camera detects the Thymio. This update compares the real measurement and the estimated one, in order to make the algorithm better (by calculating the residual covariance matrix and the Kalman gain). If the camera doesn't detect the Thymio, we use the prediction the localize the robot, that allows the robot to work even though the camera is hidden. The following formula represents the state space model that we use to at each time step :
<img src="Images/Kalman.png" alt="Drawing" style="width: 500px;"/>

### 2.4 Local Navigation

### 2.5 Motion control
<p>$\;\;\;\;\;\;$ Thymio has a constant speed but we need a motion control to ensure smooth turns and to correct the drifting or the wheels' error. A controller seems to be the best solution to these problematics.
<br>$\;\;\;\;\;\;$ Therefore, we chose to use a proportional-derivative (PD) controller to compute our rotation speed. This speed is sumed to our constant speed (actually, the roation speed is sumed to one of the wheel's speed and subtracted to other one) in a way that the robot never stops to turn. First, our idea was to use a simple proportional controller but we expected some trouble with the local navigation since Thymio has no lateral proximity sensor. Indeed, when the robot doesn't detect the local obstacle anymore, it might suddenly turn and hit the obstacle. Adding a derivative parameter permits to adjust even more the smoothness of the curves. 
<br>$\;\;\;\;\;\;$ The PD controller formula is 
<br> <h1><center>${v}_{rotation} = {K}_{p}{e}_{angle} + {K}_{d} {\Delta}_{error}$</center></h1> 
<br> Where ${e}_{angle}$ is the error between the current Thymio's angle and the angle he is supposed to have to reach its destination. ${\Delta}_{error}$ stands for the error variation between the previous state and the current one. We empirically found the proportional (Kp) and derivative (Kd) gains : ${K}_{p}$ = 12 and ${K}_{d}$ = 6.

## 3. Global Code Execution

### 3.1 Imports
<p>At first, we will start by importing the different .py files that we created and where are defined most of the functions we will use in this program :</p>

- **init_map.py** defines the functions related to a linear Kalman filter
- **motion_control.py** defines the functions related to the motion control of the robot (PD controller, local avoidance)
- **vis_graph.py** defines the functions related to the creation of the map and the visibility graph
- **update_pos.py** defines the functions that are used to detect the position and orientation of the robot with the camera
- **kalman_filter.py** defines the functions used to apply our Kalman filter to the program

In [2]:
import tdmclient.notebook
await tdmclient.notebook.start()
from tdmclient import ClientAsync, aw
client = ClientAsync()
client.process_waiting_messages()
node = await client.wait_for_node()
aw(node.run())
aw(node.stop())
aw(node.unlock())

import time
import cv2
from init_map import *
from motion_control import *
from vis_graph import *
from update_pos import *
from kalman_filter import *

Exception: Node lock error

### 3.2 Definition of getters and setters
<p>In the following section, we define the getters and setters that will allow us to communicate with the robot regarding the sensors and the motor speeds :</p>

In [1]:
@tdmclient.notebook.sync_var
def get_prox_value():
    global prox_horizontal
    return prox_horizontal

@tdmclient.notebook.sync_var
def set_motor_speed(left_speed, right_speed):
    global motor_left_target, motor_right_target
    motor_left_target = left_speed
    motor_right_target = right_speed

NameError: name 'tdmclient' is not defined

### 3.3 Initialisation of the variables and constants
<p>At first, we define several constants that will be used in the initialisation :</p>

In [ ]:
#Create constant variables
next_goal_trigger = 20
delta_t = 0.05

### 3.4 Creation of the map and the global path
<p>In this section, we will explain how the map is created with the camera at the start of the program. The following cell allows to show the shortest path computed on our map.</p>

In [ ]:
VideoCap = cv2.imread("Images/Map_init.png")

#Initialisation of the variables that will register the coordinates of the starting point and the final goal
start = []
stop = []

#Creation of the map and the global path
while(len(stop)==0 or (len(start)==0)):
    start, stop = detect_start_stop(VideoCap)

global_path = initialisation(VideoCap, start, stop)
start = global_path[0]
stop = global_path[len(global_path)-1]

### 3.5 Initialisation of the variables
<p>In this section, we will initialise the different variables that will be used during the program

<p>Finally, we can now start implementing the program on Thymio.</p>

## 4. References
- visibility graph/shortest path : https://github.com/TaipanRex/pyvisgraph/tree/master/pyvisgraph
- extended kalman filter : https://automaticaddison.com/extended-kalman-filter-ekf-with-python-code-example/
- kalman filter, l42 project : https://github.com/L42Project/Tutoriels/blob/master/Divers/tutoriel36/KalmanFilter.py
- kalman filter, wikipedia : https://en.wikipedia.org/wiki/Kalman_filter
- obstacle avoidance, TP 3 of the course : http://localhost:8906/notebooks/Ex3/Solutions%20Week%203%20-%20Artificial%20neural%20networks.ipynb
- openCV documentation : https://docs.opencv.org/4.x/d6/d00/tutorial_py_root.html